Joseph Lannan and Merve Baksi \\
10/2/20


# HW4: Monte-Carlo Integration in 1d 

The below problems are to make you acquainted with MC integration methods. In practice, MC integration is only advantage whenever one has to deal with complicated, high dimensional integrals.  


*Problem* 4 

**Read** 4.3.4; specifically pay attention to three methods of MC integration:


1.   Hit and miss method
2.   Mean-value integration
3.   General MC integration

Using the template below write code for this integration methods.  

I am assuming we want to do our integration from 0-1



In [ ]:
from random import *
import numpy as np 
import scipy.stats as stats
import scipy

def normal(mean=0, sigma=1): #whats this for?
  x1 = uniform(0,1)
  x2 = uniform(0,1)
  z1 = mean+sigma*np.cos(2*np.pi*x2)*np.sqrt(-2.0*np.log(x1))
  z2 = mean+sigma*np.cos(2*np.pi*x2)*np.sqrt(-2.0*np.log(x1))
  return (z1,z2)

def func(x):
  return np.sin(x)*np.exp(-8*(x-0.5*np.pi)**2) 

def hitandmiss(f,N):
  # Write a function performing hit and miss MC integration of f using N points 
  # The function should return the value of the integral 

  hits = 0
  size = .05 #make this bigger to integrate larger funcitons, more efficient if this is close to the max of the function
  for i in range(0,N):
    point = np.array([random(),size*random()]) #generating random point in (0-1,0-1), this will not work if our function is larger than 1 in (0,1), in that case we can 
    if point[1] < f(point[0]): #check to see if point is under f(x)
      hits += 1
  return size*hits/N


def meanvalueintegration(f,N):
  # Write a function performing mean value MC integration of f using N points
  # The function should return the value of the integral

  return sum(f(np.random.rand(N)))/N

def mcgeneral(f,N,lb,ub):
  # Write a function performing general MC integration of f using N points
  # The function should return the value of the integral
  # Here you have to use the form of the function we are integrating; 
  # for p(x) use C*np.exp(-(x-0.5*np.pi)**2/2.0/0.0625), where C is the normalization

  def foverp(x): return np.sin(x) #defining f to be sin(x) to make it faster.

  C = 1/(0.313329*scipy.special.erf(2.82843*(-1.5708 + ub)) - (0.313329*scipy.special.erf(2.82843*(-1.5708 + lb))))

  lower, upper = lb, ub 
  mu, sigma = np.pi/2, (1/4)     #p(x) is just an gaussian with mu = pi/2 and sigma = 1/4
  X = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma) #i needed something to generate random numbers in a truncated gaussian see documentation for how this works: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.truncnorm.html
  randnums = X.rvs(N)
  return sum(foverp(randnums))/N/C

Execute below after you complete the code above \\
Note: actual is .00554756 \\
https://www.wolframalpha.com/input/?i=integral+from+0+to+1+of+sin%28x%29*exp%28-8*%28x-0.5*pi%29**2%29

In [ ]:
Nofpoints = 100 
tol = 1 

y1 = hitandmiss(func,Nofpoints)

template = "{0:20}|{1:20}|{2:22}" 
print (template.format("# opf points", "Result", "Tolerance")) # header
  
while tol > 1e-4: 
  y2 = hitandmiss(func, 2 * Nofpoints)
  tol = abs(y2-y1)#this error check can get messed up if the two number just so happen to be near each other, perhaps we also want to check the one before to be more certain
  print (template.format(Nofpoints, y1, tol))
  Nofpoints = 2 * Nofpoints
  y1 = y2

print('_'*64)
print('Hit and miss: result is ', y1)

# opf points        |Result              |Tolerance             
                 100|0.0030000000000000005| 0.0030000000000000005
                 200|0.006000000000000001| 0.0010000000000000009
                 400|               0.005|0.00012499999999999924
                 800|0.004875000000000001| 0.0013437499999999995
                1600|          0.00621875| 0.0005468749999999996
                3200|0.005671875000000001|0.00020312500000000105
                6400|          0.00546875| 2.343750000000002e-05
________________________________________________________________
Hit and miss: result is  0.0054921875


In [ ]:
Nofpoints = 100
tol = 1 

y1 = meanvalueintegration(func,Nofpoints)

template = "{0:20}|{1:20}|{2:22}" 
print (template.format("# opf points", "Result", "Tolerance")) # header
  
while tol > 1e-4: 
  y2 = meanvalueintegration(func, 2 * Nofpoints)
  tol = abs(y2-y1)
  print (template.format(Nofpoints, y1, tol))
  Nofpoints = 2 * Nofpoints
  y1 = y2

print('_'*64)
print('Mean value: result is ', y1)

# opf points        |Result              |Tolerance             
                 100| 0.00664932622084184| 0.0006376950063099367
                 200|0.007287021227151777| 0.0020856051361233625
                 400|0.005201416091028414| 0.0001382077348334558
                 800| 0.00533962382586187| 0.0002755926824846329
                1600|0.005615216508346503|1.4215325095479764e-05
________________________________________________________________
Mean value: result is  0.005629431833441983


In [ ]:
Nofpoints = 100
tol = 1 

lowerbound = 0
upperbound = 1

y1 = mcgeneral(func,Nofpoints,lowerbound,upperbound)

template = "{0:20}|{1:20}|{2:22}" 
print (template.format("# opf points", "Result", "Tolerance")) # header
  
while tol > 1e-4: 
  y2 = mcgeneral(func,2 * Nofpoints,lowerbound,upperbound)
  tol = abs(y2-y1)
  print (template.format(Nofpoints, y1, tol))
  Nofpoints = 2 * Nofpoints
  y1 = y2

print('_'*64)
print('MC general method: result is ', y1)

# opf points        |Result              |Tolerance             
                 100|0.005575831775568661| 4.831591182017855e-05
________________________________________________________________
MC general method: result is  0.005527515863748483


In [ ]:
Nofpoints = 100
tol = 1 

lowerbound = 0
upperbound = np.pi

y1 = mcgeneral(func,Nofpoints,lowerbound,upperbound)

template = "{0:20}|{1:20}|{2:22}" 
print (template.format("# opf points", "Result", "Tolerance")) # header
  
while tol > 1e-4: 
  y2 = mcgeneral(func,2 * Nofpoints,lowerbound,upperbound)
  tol = abs(y2-y1)
  print (template.format(Nofpoints, y1, tol))
  Nofpoints = 2 * Nofpoints
  y1 = y2

print('_'*64)
print('MC general method: result is ', y1)

# opf points        |Result              |Tolerance             
                 100|  0.6048357957587176|  0.003027155744394605
                 200|  0.6078629515031122|2.8255302850110908e-05
________________________________________________________________
MC general method: result is  0.6078346962002621


Note: I asked during class and Dr. Skokov said to change the bounds from 0 to pi

for 0 to pi, actual should be 0.607377 \\
https://www.wolframalpha.com/input/?i=integral+from+0+to+pi+of+sin%28x%29*exp%28-%28x-0.5*pi%29**2%2F2.0%2F0.0625%29

for 0 to 1, actual should be .00554756 \\
https://www.wolframalpha.com/input/?i=integral+from+0+to+1+of+sin%28x%29*exp%28-%28x-0.5*pi%29**2%2F2.0%2F0.0625%29